In [1]:
import matplotlib.pyplot as plt

from astropy.io import fits

import numpy as np

import matplotlib.colors as colors

In [28]:
with fits.open('/home/kevin/Documents/N1333/N-A-L671/no_bias/SUPA01425775.fits') as f:
    img_hdr = f[0].header.copy()
    img_data = f[0].data.copy()

img_data = img_data.byteswap().newbyteorder()


In [4]:
import sep
sep.get_sub_object_limit()

1024

In [60]:
from ccdproc import cosmicray_lacosmic

In [19]:
def df2reg(df, regpath, name_root='Region'):
    reg_hdr = ['# Region file format: DS9 version 4.1',
               'global color=red dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1',
               'physical']
    hasnames = 'name' in df.columns
    if hasnames:
        names = df.name
    else:
        names = [f'{name_root}_{i:02d}' for i in range(len(df))]

    with open(regpath,'w') as regf:
        for h in reg_hdr:
            regf.write(h+'\n')
    
        for name, reg in zip(names, df.itertuples()):
            nametxt = '{' + name + '}'
            reg_str = f'circle({reg.x_pos}, {reg.y_pos}, {reg.radius}) # text={nametxt}'
            regf.write(reg_str+'\n')

In [73]:
import pandas as pd
def find_stars(fitsin,  regout, thresh = 50):
    with fits.open(fitsin) as f:
        img_hdr = f[0].header.copy()
        img_data = f[0].data.copy()

    img_data = img_data.byteswap().newbyteorder()
    img_bkg = sep.Background(img_data)
    bkg_img =img_bkg.back()
    img_sub = img_data - bkg_img
    img_sub, mask = cosmicray_lacosmic(img_sub)
    objects = sep.extract(img_sub, thresh, err=img_bkg.globalrms)
    print(f'Number of objects identified: {len(objects)}')
    objects_df = pd.DataFrame(objects)
    reg_df = pd.DataFrame({'x_pos':objects_df.x+1, 'y_pos':objects_df.y+1,'radius':8})
    df2reg(reg_df, regout)

In [29]:
img_bkg = sep.Background(img_data)
img_bkg.globalback, img_bkg.globalrms

(3058.485595703125, 57.94922637939453)

In [61]:
bkg_img =img_bkg.back()
img_sub = img_data - bkg_img
img_sub, mask = cosmicray_lacosmic(img_sub)
objects = sep.extract(img_sub, 50, err=img_bkg.globalrms)

len(objects)

75

In [ ]:
import pandas as pd
objects_df = pd.DataFrame(objects)
objects_df

In [62]:
#stars = np.logical_and(objects_df.npix >= 150, objects_df.npix < 300)
reg_df = pd.DataFrame({'x_pos':objects_df.x+1, 'y_pos':objects_df.y+1,'radius':8})
df2reg(reg_df, '/home/kevin/Documents/N1333/N-A-L671/sep.reg')

In [27]:
objects_df.columns

Index(['thresh', 'npix', 'tnpix', 'xmin', 'xmax', 'ymin', 'ymax', 'x', 'y',
       'x2', 'y2', 'xy', 'errx2', 'erry2', 'errxy', 'a', 'b', 'theta', 'cxx',
       'cyy', 'cxy', 'cflux', 'flux', 'cpeak', 'peak', 'xcpeak', 'ycpeak',
       'xpeak', 'ypeak', 'flag'],
      dtype='object')

In [63]:
objects_df.describe()

,thresh,npix,tnpix,xmin,xmax,ymin,ymax,x,y,x2,...,cxy,cflux,flux,cpeak,peak,xcpeak,ycpeak,xpeak,ypeak,flag
count,104.000000,104.000000,104.000000,104.000000,104.000000,104.000000,104.000000,104.000000,104.000000,104.000000,...,104.000000,1.040000e+02,1.040000e+02,104.000000,104.000000,104.000000,104.000000,104.000000,104.000000,104.000000
mean,2897.461426,284.836538,256.230769,876.692308,901.971154,1961.028846,1988.759615,894.100837,1976.182369,213.244168,...,-0.379554,1.791711e+07,1.806350e+07,26372.571007,30650.353952,894.182692,1978.144231,894.201923,1978.096154,1.288462
std,0.000000,2168.834782,1933.323986,719.314885,722.515004,1289.698305,1314.560626,716.672072,1301.978331,2133.980980,...,1.714196,1.708514e+08,1.720552e+08,49085.602966,49544.622691,716.913771,1304.032919,716.898679,1303.794816,2.945041
min,2897.461426,5.000000,3.000000,0.000000,0.000000,0.000000,6.000000,0.000000,2.471942,0.071513,...,-8.301023,1.460816e+04,2.268148e+04,2931.119141,4249.408203,0.000000,1.000000,0.000000,0.000000,0.000000
25%,2897.461426,7.000000,6.000000,248.500000,267.500000,914.500000,918.750000,263.769528,916.847556,0.379805,...,-0.052954,2.458257e+04,3.459100e+04,4317.448486,7010.601562,263.500000,917.000000,263.500000,917.250000,0.000000
50%,2897.461426,16.000000,13.000000,676.000000,712.000000,1833.000000,1839.000000,709.626298,1836.428800,0.905036,...,-0.005217,6.354279e+04,7.651195e+04,5889.029053,10637.668945,709.500000,1836.500000,709.500000,1836.500000,0.000000
75%,2897.461426,66.000000,61.000000,1581.750000,1723.000000,2979.000000,3209.500000,1586.547414,3103.721519,3.253009,...,0.002839,6.498321e+05,6.640850e+05,17685.601074,22344.247559,1589.750000,3207.750000,1589.000000,3207.000000,1.000000
max,2897.461426,22142.000000,19741.000000,2047.000000,2047.000000,4160.000000,4176.000000,2047.000000,4163.008529,21766.088943,...,3.113304,1.743193e+09,1.755498e+09,198094.218750,198102.562500,2047.000000,4163.000000,2047.000000,4163.000000,10.000000


In [78]:
from ccdproc import ImageFileCollection
import os, shutil

dirname = '/home/kevin/Documents/N1333/N-A-L671/'
regdir = os.path.join(dirname, 'region')
try:
    shutil.rmtree(regdir)
except:
    pass
os.mkdir(regdir)
no_bias_ifc = ImageFileCollection(os.path.join(dirname, 'no_bias'))
for nbf in no_bias_ifc.files_filtered(include_path=True):
    regname = os.path.splitext(os.path.basename(nbf))[0] + '.reg'
    regpath = os.path.join(regdir, regname)
    print(f'file: {nbf}, region: {regpath}')
    find_stars(nbf, regpath)

file: /home/kevin/Documents/N1333/N-A-L671/no_bias/SUPA01425770.fits, region: /home/kevin/Documents/N1333/N-A-L671/region/SUPA01425770.reg
Number of objects identified: 69
file: /home/kevin/Documents/N1333/N-A-L671/no_bias/SUPA01425771.fits, region: /home/kevin/Documents/N1333/N-A-L671/region/SUPA01425771.reg
Number of objects identified: 29
file: /home/kevin/Documents/N1333/N-A-L671/no_bias/SUPA01425772.fits, region: /home/kevin/Documents/N1333/N-A-L671/region/SUPA01425772.reg
Number of objects identified: 48
file: /home/kevin/Documents/N1333/N-A-L671/no_bias/SUPA01425773.fits, region: /home/kevin/Documents/N1333/N-A-L671/region/SUPA01425773.reg
Number of objects identified: 114
file: /home/kevin/Documents/N1333/N-A-L671/no_bias/SUPA01425774.fits, region: /home/kevin/Documents/N1333/N-A-L671/region/SUPA01425774.reg
Number of objects identified: 27
file: /home/kevin/Documents/N1333/N-A-L671/no_bias/SUPA01425775.fits, region: /home/kevin/Documents/N1333/N-A-L671/region/SUPA01425775.reg


In [75]:
xx = np.arange(56).astype(np.float32).reshape(7,8)
xx

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11., 12., 13., 14., 15.],
       [16., 17., 18., 19., 20., 21., 22., 23.],
       [24., 25., 26., 27., 28., 29., 30., 31.],
       [32., 33., 34., 35., 36., 37., 38., 39.],
       [40., 41., 42., 43., 44., 45., 46., 47.],
       [48., 49., 50., 51., 52., 53., 54., 55.]], dtype=float32)

In [77]:
xx[:2]=np.nan
xx[-2:] = np.nan
xx[:,:2]=np.nan
xx[:,-2:] = np.nan
xx

array([[nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, 18., 19., 20., 21., nan, nan],
       [nan, nan, 26., 27., 28., 29., nan, nan],
       [nan, nan, 34., 35., 36., 37., nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan]], dtype=float32)